In [ ]:
import s3fs
import nibabel as nib
from nibabel.affines import rescale_affine
import pandas as pd 
import matplotlib.pyplot as plt 
from skimage.transform import rescale
from skimage.filters import gaussian
from nilearn import plotting 
import numpy as np

In [ ]:
fs = s3fs.S3FileSystem(anon=True)

In [ ]:
mkdir derivatives

In [ ]:
subject = 3 #change subject number to run 
trials_info = pd.read_csv('../annotation_data/shared_trial_info.csv')
trials_sub = trials_info[((trials_info.SUBJECT == subject))]
trials_sub.head()
len(trials_sub)

In [ ]:
#to parallelize, first make a function

#toggle i to run 1 or all sessions (40 sessions total) 
i = 0 #0 = run all sessions, 1 = run a single session 
if i == 0:
    sessions = trials_sub.SESSION.unique() #to loop over all sessions
else: 
    sessions = [1] #change number to session you want to run 

for session in sessions:
    if (session < 10):
        ll = fs.ls(f'natural-scenes-dataset/nsddata_betas/ppdata/subj0{subject}/MNI/betas_fithrf/betas_session0{session}.nii.gz')
    else: 
        ll = fs.ls(f'natural-scenes-dataset/nsddata_betas/ppdata/subj0{subject}/MNI/betas_fithrf/betas_session{session}.nii.gz')
    fs.get(ll[0], "tmp.nii.gz")   
    img = nib.load("tmp.nii.gz")
    trials_sub_ses = trials_sub[(trials_sub.SESSION == session)]
    print("file loaded")
    
    for count, t in enumerate(trials_sub_ses['trial_in_session'].tolist()):
        beta_trial = img.slicer[:,:,:,t].get_fdata()/300 #get trial 1
                    
        #smooth
        img_smoothed = gaussian(beta_trial, sigma=4)

        #downsample 
        img_rescaled = rescale(img_smoothed, 0.25, anti_aliasing=False)

        rescaled_affine = rescale_affine(img.affine, shape = beta_trial.shape, zooms = (4, 4, 4), new_shape = img_rescaled.shape)

        #flattening 
        flat_img = img_rescaled.flatten()
        
        if (count ==0):
            ses_betas = flat_img
        else:
            ses_betas = np.vstack((ses_betas, flat_img))
        print(f"ran trial {t} for session {session}")
    #save to np array 
    fname = f"./derivatives/subj{subject}_ses{session}.npy"
    np.save(fname, ses_betas)
    print(f"saved file for session {session}")

In [ ]:
# overlay beta map with brain mask
plotting.plot_stat_map(nib.Nifti1Image(img_rescaled, rescaled_affine), # this function needs a 3D data file
                       cut_coords=(0, 0, 0), #use to select coordinates that you want to view
                       title="test",
                       vmax=5)